In [1]:
import dgl
import numpy as np
import networkx as nx
from node2vec import Node2Vec
import matplotlib.pyplot as plt
from operator import itemgetter
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.neighbors import NearestNeighbors
import itertools

import dgl.data
import dgl.function as fn
import dgl.nn.pytorch as dglnn
from dgl.nn import GraphConv
from dgl.nn import SumPooling
from dgl.nn import DenseGraphConv
from dgl.nn import SAGEConv

from tqdm import tqdm
import time

Using backend: pytorch


In [2]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super().__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]
    

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']
        
class FC(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats*2, h_feats*2)
        self.W2 = nn.Linear(h_feats*2, h_feats)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']
        
def positive_sample(graph, test_size=0.1):
    u, v = graph.edges()
    eids = np.random.permutation(np.arange(graph.number_of_edges())) #random index edges
    test_size_idx = int(len(eids) * test_size) #size positive sample by index

    test_pos_u, test_pos_v = u[eids[:test_size_idx]], v[eids[:test_size_idx]]
    train_pos_u, train_pos_v = u[eids[test_size_idx:]], v[eids[test_size_idx:]] 
    
    train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=graph.number_of_nodes())
    test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=graph.number_of_nodes())
    
    return train_pos_g, test_pos_g, eids


def negative_sample(graph, method='kneighbors', size=None, test_size=None):   
    new_g = graph.to_networkx()
    adj = nx.to_numpy_array(new_g) #adjacency matrix
    
    if size == None:
        size=g.number_of_nodes()
    if test_size == None:
        test_size=int(g.number_of_edges()*0.1)
    
    if method == 'dgl_example':
        adj_neg = 1 - adj - np.eye(graph.number_of_nodes())
        neg_u, neg_v = np.where(adj_neg != 0)
        neg_eids = np.random.choice(len(neg_u), graph.number_of_edges() // 2) #negative sample random index
        
        test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
        train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]
    
    else:
        negs_u = []
        negs_v = []
        negs = []
        nnn = NearestNeighbors(n_neighbors=500, metric='cosine')
        nnn.fit(adj)
        res = nnn.kneighbors(return_distance=False) #top-5 nearest neightbord

        for idx, i in enumerate(res):
            for j in i:
                if not new_g.has_edge(idx, j):
                    negs.append([idx, j])

        negs = np.array(negs)

        for k in range(size):
            temp = negs[np.random.permutation(negs.shape[0])[:graph.number_of_edges()]][0]
            negs_u.append(temp[0])
            negs_v.append(temp[1])
            
        test_neg_u, test_neg_v = negs_u[:test_size], negs_v[:test_size]
        train_neg_u, train_neg_v = negs_u[test_size:], negs_v[test_size:]
    
    train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=graph.number_of_nodes())
    test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=graph.number_of_nodes())
            
    return train_neg_g, test_neg_g


def alternate_list(a,b):
    c = list()
    for x in range(len(a)):
        c.extend([a[x], b[x]])
    return c

In [32]:
# create random graph
spmat = sp.rand(100, 100, density=0.01) # 5% nonzero entries
g = dgl.from_scipy(spmat)
g

Graph(num_nodes=100, num_edges=100,
      ndata_schemes={}
      edata_schemes={})

In [33]:
# dataset = dgl.data.CoraGraphDataset()
# g = dataset[0]

# g = dgl.remove_edges(g, eids[:1000], ) #subgraph
# g = dgl.remove_nodes(g, range(2000))
# print(g)

# G = g.to_networkx()

# print(len(list(G.nodes())))
# print(len(list(G.edges())))

In [34]:
# train-test and positive-negative sampling
train_pos_g, test_pos_g, eids = positive_sample(g)
train_neg_g, test_neg_g = negative_sample(g, 'dgl_example')
###########
print('train_pos_shape =', [len(train_pos_g.nodes()), len(train_pos_g.edges()[0])], '; test_pos_shape =', [len(test_pos_g.nodes()), len(test_pos_g.edges()[0])])
print('train_neg_shape =', [len(train_neg_g.nodes()), len(train_neg_g.edges()[0])], '; test_neg_shape =', [len(test_neg_g.nodes()), len(test_neg_g.edges()[0])])

train_pos_shape = [100, 90] ; test_pos_shape = [100, 10]
train_neg_shape = [100, 40] ; test_neg_shape = [100, 10]


In [35]:
G = train_pos_g.to_networkx()
print(len(list(G.nodes())))
print(len(list(G.edges())))

100
90


In [36]:
# node2vec parameters 
dimensions=10
walk_length=10
window=10
min_count=1

In [37]:
# create node features
node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length)#, workers=6)
model_n2v = node2vec.fit(window=window, min_count=min_count)
embeddings = np.array([model_n2v.wv[x] for x in list(G.nodes())])
embeddings = torch.from_numpy(embeddings)
g.ndata['feat'] = embeddings                       # f_u

Computing transition probabilities:   0%|          | 0/100 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 1073.21it/s]


In [38]:
#list(embeddings[:5] - g.ndata['feat'][:5])

In [39]:
test_size=int(g.number_of_edges()*0.1)
print('test_size =', test_size)
train_g = dgl.remove_edges(g, eids[:test_size])
train_g

test_size = 10


Graph(num_nodes=100, num_edges=90,
      ndata_schemes={'feat': Scheme(shape=(10,), dtype=torch.float32)}
      edata_schemes={})

### line graph

In [40]:
# create line graph 
temp_G = nx.Graph()
temp_G.add_edges_from(list(G.edges()))
LnxG = nx.line_graph(temp_G)

if len(list(LnxG.nodes())) != len(list(G.edges())):
    print('diffrent number of G.nodes LnxG.nodes! Used DiGraph.')
    temp_G = nx.DiGraph()
    temp_G.add_edges_from(list(G.edges()))
    LnxG = nx.line_graph(temp_G)

diffrent number of G.nodes LnxG.nodes! Used DiGraph.


In [41]:
print(len(list(G.edges())), sorted(list(G.edges()),reverse=False)[:10])

90 [(0, 86), (1, 37), (1, 80), (2, 18), (2, 36), (5, 13), (6, 47), (8, 53), (8, 76), (10, 19)]


In [42]:
print(len(list(LnxG.nodes())), sorted(list(LnxG.nodes()),reverse=False)[:10])
print(len(list(LnxG.edges())), list(LnxG.edges())[:10])

90 [(0, 86), (1, 37), (1, 80), (2, 18), (2, 36), (5, 13), (6, 47), (8, 53), (8, 76), (10, 19)]
81 [((0, 86), (86, 69)), ((86, 69), (69, 13)), ((69, 13), (13, 45)), ((2, 18), (18, 93)), ((5, 13), (13, 45)), ((13, 45), (45, 13)), ((45, 13), (13, 45)), ((8, 76), (76, 68)), ((8, 53), (53, 10)), ((8, 53), (53, 59))]


In [43]:
# lg - line graph on DGL; LG - line grpah on nx
# ф-ция, которая переназывает вершины и ребра дуального графа
# если вершина была  (0, 633), то может стать вершиной 5, например
def create_dgl_nx_dual_graph(line_nx_graph):
    nodes = sorted(list(line_nx_graph.nodes()),reverse=False)
    edges = list(line_nx_graph.edges())
    nodes_dict = {}
    new_u, new_v = [], []
    
    for idx, val in enumerate(nodes):
        nodes_dict[val] = idx
    
    print(len(nodes_dict), len(edges))
    for edge in edges:
        new_u.append(nodes_dict[edge[0]])
        new_v.append(nodes_dict[edge[1]])
    
    u = torch.tensor(new_u)
    v = torch.tensor(new_v)
    g = dgl.graph((u, v))
    G = g.to_networkx()
    return g, G
    
    
lg, LG = create_dgl_nx_dual_graph(LnxG)
dual_edges_dict = {edge: num for num, edge in enumerate(list(LG.edges()))}
dual_nodes_dict = {node: num for num, node in enumerate(sorted(list(LnxG.nodes()),reverse=False))}

90 81


In [44]:
#print(len(dual_nodes_dict))
#print(len(dual_edges_dict))
print(len(list(LG.nodes)), list(LG.nodes)[:10])
print(len(list(LG.edges)))
print(list(LG.edges())[:10])

90 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
81
[(0, 78), (3, 20), (5, 16), (7, 44), (7, 46), (7, 45), (8, 69), (12, 67), (13, 22), (14, 66)]


In [45]:
print(len(list(lg.nodes())), list(lg.nodes())[:10])
print(len(list(lg.edges())[0]))
print(list(lg.edges())[0][:10], list(lg.edges())[0][:10])

90 [tensor(0), tensor(1), tensor(2), tensor(3), tensor(4), tensor(5), tensor(6), tensor(7), tensor(8), tensor(9)]
81
tensor([ 0, 78, 64,  3,  5, 16, 37,  8,  7,  7]) tensor([ 0, 78, 64,  3,  5, 16, 37,  8,  7,  7])


In [46]:
# create node features to line graph

# m = nn.AvgPool1d(2, stride=2)
node2vec = Node2Vec(LnxG, dimensions=dimensions, walk_length=walk_length)
model_n2v_dual = node2vec.fit(window=window, min_count=min_count)
#embeddings_dual = [[alternate_list(model_n2v_dual.wv[x][0],model_n2v_dual.wv[x][1]) for x in list(LnxG.nodes)]]
embeddings_dual = [model_n2v_dual.wv[x] for x in list(LG.nodes)]
#embeddings_dual = m(torch.tensor(embeddings_dual))[0]
#embeddings_dual = (torch.tensor(embeddings_dual))[0]
embeddings_dual = torch.tensor(embeddings_dual)
lg.ndata['feat'] = embeddings_dual                #f_uv^*

Computing transition probabilities:   0%|          | 0/90 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 888.57it/s]
/var/folders/cm/gfrlqbxx32b2v0hgshlc8mk00000gn/T/ipykernel_95899/3184879234.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  embeddings_dual = torch.tensor(embeddings_dual)


In [47]:
#list(embeddings_dual[:5] - lg.ndata['feat'][:5])

In [48]:
train_dual_g = lg #dgl.remove_edges(dual_g, eids[:int(len(dual_eids) * 0.1)]) #subgraph
print(train_dual_g)

Graph(num_nodes=90, num_edges=81,
      ndata_schemes={'feat': Scheme(shape=(10,), dtype=torch.float32)}
      edata_schemes={})


In [120]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], dimensions)
FC_net = FC(dimensions)
###############################################################
model_dual = GraphSAGE(train_dual_g.ndata['feat'].shape[1], dimensions)
FC_net_dual = FC(dimensions)

In [121]:
def g_u_star(G, LnxG, pos_score_dual):
    node_features = np.zeros((G.number_of_nodes(), dimensions))
    counts = np.zeros((G.number_of_nodes(), 1))

    for node in list(G.nodes()):
        for i, edge in enumerate(list(LnxG.edges())):
            if (node in edge[0])or(node in edge[1]):
                n1, n2 = dual_nodes_dict[edge[0]], dual_nodes_dict[edge[1]]
                #print(edge, n1, n2)
                try:
                    num_embd_edge_G_star = dual_edges_dict[(n1, n2)]
                    node_features[node] += pos_score_dual[num_embd_edge_G_star].detach().numpy()
                    counts[node] += 1
                    #print(pos_score_dual[num_embd_edge_G_star])
                except: print('NUN', edge)
        #print(counts)
        if counts[node] != 0:
            node_features[node]/=counts[node]
        
    return torch.from_numpy(node_features)

In [122]:
def compute_loss_for_train_SAGE(alpha, beta, z_u, g_u_star, g_uv, z_uv_star):
    res1 = alpha*((z_u - g_u_star)**2).mean()
    res2 = beta*((g_uv - z_uv_star)**2).mean()
    return res1 + res2 
    # return alfa*((z_u - g_u_star)**2).mean() + beta*((g_uv - z_uv_star)**2).mean()
    #return alfa*F.binary_cross_entropy_with_logits(z_u, g_u_star)

In [135]:
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), FC_net.parameters()), lr=0.01)
optimizer_d = torch.optim.Adam(itertools.chain(model_dual.parameters(), FC_net_dual.parameters()), lr=0.01)

all_logits, diff = [], 0
alpha, beta = 0.4, 0.9

for e in tqdm(range(1000)):
    h = model(train_g, embeddings)                              #z_u
    h_dual = model_dual(train_dual_g, embeddings_dual)          #z_uv^*
    
    pos_score = FC_net(train_g, h)                              #g_uv
    #neg_score = pred(train_neg_g, h)                           #g_uv -
    pos_score_dual = FC_net_dual(train_dual_g, h_dual)          #g_u^*
    #neg_score_dual = pred(train_neg_dual_g, h_dual)            #g_u^* -
    start = time.time()
    g_u_s = g_u_star(G, LnxG, pos_score_dual)
    end = time.time()
    diff += int(end - start)
    loss = compute_loss_for_train_SAGE(alpha, beta, h, g_u_s, pos_score, h_dual)
    
    # print(h.shape)
    # print(h_dual.shape)
    # print(pos_score.shape)
    # print(pos_score_dual.shape)
    # print(g_u_s.shape)
    # print(loss, F.mse_loss(h, g_u_s), F.mse_loss(pos_score, h_dual))
    
    optimizer.zero_grad()
    optimizer_d.zero_grad()
    loss.backward()
    optimizer.step()
    optimizer_d.step()

    if e % 40 == 0:
        print('In epoch {}, loss: {}, time_g_u_s: {} s'.format(e, loss, diff))
        diff = 0
    
# torch.Size([7, 3])
# torch.Size([8, 3])
# torch.Size([8, 3])
# torch.Size([12, 3])

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
  2%|▏         | 17/1000 [00:00<00:11, 83.37it/s]

In epoch 0, loss: 0.00018082465955771674, time_g_u_s: 0 s


  5%|▌         | 54/1000 [00:00<00:10, 89.12it/s]

In epoch 40, loss: 0.00017779051883315492, time_g_u_s: 0 s


  9%|▉         | 92/1000 [00:01<00:10, 90.54it/s]

...

 93%|█████████▎| 932/1000 [00:10<00:00, 92.18it/s]

In epoch 920, loss: 0.00016276756671540694, time_g_u_s: 0 s


 97%|█████████▋| 972/1000 [00:10<00:00, 92.51it/s]

In epoch 960, loss: 0.00016270205210827261, time_g_u_s: 0 s


100%|██████████| 1000/1000 [00:11<00:00, 89.40it/s]


# LP Task 

In [136]:
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])    
    return F.binary_cross_entropy_with_logits(scores, labels)
    #return ((scores - labels)**2).mean() 

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [137]:
model_lp = GraphSAGE(train_g.ndata['feat'].shape[1], 20)
pred_lp = MLPPredictor(dimensions)
#pred_lp = DotPredictor()

In [138]:
print(train_pos_g)
print(train_neg_g)

Graph(num_nodes=100, num_edges=90,
      ndata_schemes={}
      edata_schemes={})
Graph(num_nodes=100, num_edges=40,
      ndata_schemes={}
      edata_schemes={})


In [139]:
optimizer_lp = torch.optim.Adam(pred_lp.parameters(), lr=0.01)

all_logits = []
for e in range(200):
    model.eval()
    h_after_GCN = model(train_g, embeddings)  #train_g.ndata['feat'])
    pos_score = pred_lp(train_pos_g, h_after_GCN)
    neg_score = pred_lp(train_neg_g, h_after_GCN)
    loss_lp = compute_loss(pos_score, neg_score)
    
    #print(loss_lp)
    
    optimizer_lp.zero_grad()
    loss_lp.backward(retain_graph=True)
    optimizer_lp.step()

    if e % 10 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))


from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred_lp(test_pos_g, h)
    neg_score = pred_lp(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.00016302126525493215
In epoch 10, loss: 0.00016302126525493215
In epoch 20, loss: 0.00016302126525493215
In epoch 30, loss: 0.00016302126525493215
In epoch 40, loss: 0.00016302126525493215
In epoch 50, loss: 0.00016302126525493215
In epoch 60, loss: 0.00016302126525493215
In epoch 70, loss: 0.00016302126525493215
In epoch 80, loss: 0.00016302126525493215
In epoch 90, loss: 0.00016302126525493215
In epoch 100, loss: 0.00016302126525493215
In epoch 110, loss: 0.00016302126525493215
In epoch 120, loss: 0.00016302126525493215
In epoch 130, loss: 0.00016302126525493215
In epoch 140, loss: 0.00016302126525493215
In epoch 150, loss: 0.00016302126525493215
In epoch 160, loss: 0.00016302126525493215
In epoch 170, loss: 0.00016302126525493215
In epoch 180, loss: 0.00016302126525493215
In epoch 190, loss: 0.00016302126525493215
AUC 0.45


In [140]:
##################################################################################################################################################################

In [141]:
print('alpha, beta =', alpha, ',',beta, '-> AUC', compute_auc(pos_score, neg_score))

alpha, beta = 0.4 , 0.9 -> AUC 0.45


In [40]:
# alpha, beta = 0.5, 1.0 -> AUC 0.55
# alpha, beta = 0.25, 1.0 -> AUC 0.525
# alpha, beta = 0.1, 1.0 -> AUC 0.5

# alpha, beta = 0.9, 0.9 -> AUC 0.48
# alpha, beta = 1.0, 0.5 -> AUC 0.48
# alpha, beta = 1.0, 0.25 -> AUC 0.45

# alpha, beta = 0.5, 0.9 -> AUC 0.54
# alpha, beta = 0.5, 0.5 -> AUC 0.59
# alpha, beta = 0.1, 0.5 -> AUC 0.29

In [ ]:
# alpha, beta = 0.5, 1. -> AUC 0.7399